<a href="https://colab.research.google.com/github/mgcvale/pytorch-playground/blob/main/iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch
import pandas as pd

In [13]:
# architecture:

"""
Dense(12, input_dim=x_train.shape[1] -> 4),
ReLU(),
Dense(6),
ReLU(),
Dense(3),
Softmax()
"""

class Model(nn.Module):
  def __init__(self, in_features=4, h1=12, h2=6, out_features=3):
    super().__init__()
    self.fc1 = nn.Linear(in_features, h1) # first layer (input to 12)
    self.fc2 = nn.Linear(h1, h2) # second layer (12 to 6)
    self.out = nn.Linear(h2, out_features) # third layer (6 to 3 softmax); softmax is implicit

  def forward(self, x):
    # feed through the layers
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x


In [58]:

class EarlyStopping:
    def __init__(self, model, save_best_weights=True, patience=6, min_delta=0.01, mode='min', min_epochs=5):
        if mode not in {"min", "max"}:
            raise ValueError(f"Invalid mode '{mode}'. Must be 'min' or 'max'.")
        if model is None:
            raise ValueError("The model must be provided.")
        if patience <= 0:
            raise ValueError("Patience must be at least 1.")

        self.model = model
        self.save_best_weights = save_best_weights
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.best_score = None
        self.counter = 0
        self.best_weights = None
        self.best_epoch = -1
        self.current_epoch = 0
        self.min_epochs = min_epochs
        self.should_stop = False  # Flag to indicate early stopping

    def step(self, current_score):
        self.current_epoch += 1
        if self.best_score is None:
            improved = True
        elif self.mode == 'min':
            improved = current_score < self.best_score - self.min_delta
        else:  # mode == 'max'
            improved = current_score > self.best_score + self.min_delta

        if improved:
            self.best_score = current_score
            if self.save_best_weights:
                self.best_weights = copy.deepcopy(self.model.state_dict())
                self.best_epoch = self.current_epoch
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience and self.current_epoch >= self.min_epochs:
                self.should_stop = True  # Set flag instead of loading weights
        return self.should_stop

In [4]:
df = pd.read_csv('Iris.csv')
df.drop('Id', inplace=True, axis=1)
df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [5]:
x = df.drop('Species', axis=1)
y = pd.Categorical(df['Species']).codes

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled = scaler.fit_transform(x.values)
x = pd.DataFrame(scaled, columns=x.columns)

In [7]:
x = x.values

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [62]:
# Now that the data is all good to go, we can define the loss function (crossentropyloss due to multiclass classification), optimizer, lr and other hyperparameters
from sklearn.metrics import f1_score

torch.autograd.set_detect_anomaly(True)

model = Model()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0015)
early_stopping = EarlyStopping(model, save_best_weights=True, patience=6, min_delta=0.012, mode='min', min_epochs=40)

epochs = 300
losses = []

# actually train the model
for i in range(epochs):
  # forward pass
  model.train()
  y_pred = model(x_train)
  loss = criterion(y_pred, y_train)


  losses.append(loss.item())

  with torch.no_grad():
    pred_classes = torch.argmax(y_pred, dim=1).cpu().numpy()
    true_classes = y_train.cpu().numpy()
    f1 = f1_score(true_classes, pred_classes, average='weighted')

  print(f"[EPOCH {i}] - Loss: {loss} - F1: {f1}")

  if early_stopping.step(loss.item()):
      print(f"[EPOCH {i}] - STOPPED DUE TO EARLY STOPPING; BEST EPOCH WAS {early_stopping.best_epoch}")
      break

  optimizer.zero_grad() # reset the error gradient from the previous backpropagation
  loss.backward() # calculate the new error gradient through backpropagation
  optimizer.step() # update model params based on backpropagation gradient


if early_stopping.best_weights is not None:
    model.load_state_dict(early_stopping.best_weights)
    print(f"Loaded best weights from epoch {early_stopping.best_epoch}")

model.eval()
# see the final accuracy on the test dataset
with torch.no_grad():
  y_test_pred = model.forward(x_test)
  pred_classes = torch.argmax(y_test_pred, dim=1).cpu().numpy()

  f1 = f1_score(y_test, pred_classes, average='weighted')
  true_classes = y_test.cpu().numpy()
  loss = criterion(y_test_pred, y_test)

print(f"\TEST PREDICTIONS: {pred_classes}, TRUE TEST VALUES: {y_test}")
print(f"FINAL LOSS: {loss} - FINAL F1: {f1}")


[EPOCH 0] - Loss: 1.1309696435928345 - F1: 0.3077717019822283
[EPOCH 1] - Loss: 1.1286275386810303 - F1: 0.3194451871657754
[EPOCH 2] - Loss: 1.1263309717178345 - F1: 0.3194451871657754
[EPOCH 3] - Loss: 1.124078392982483 - F1: 0.31823490378234903
[EPOCH 4] - Loss: 1.1218390464782715 - F1: 0.3475939269171384
[EPOCH 5] - Loss: 1.1196067333221436 - F1: 0.36583333333333334
[EPOCH 6] - Loss: 1.1173896789550781 - F1: 0.38313895781637713
[EPOCH 7] - Loss: 1.1151710748672485 - F1: 0.3927128427128427
[EPOCH 8] - Loss: 1.1129653453826904 - F1: 0.40482165404040404
[EPOCH 9] - Loss: 1.1107747554779053 - F1: 0.4364966299019608
[EPOCH 10] - Loss: 1.1085807085037231 - F1: 0.4586046948356807
[EPOCH 11] - Loss: 1.106361746788025 - F1: 0.4624226842536701
[EPOCH 12] - Loss: 1.104142665863037 - F1: 0.4676277802327294
[EPOCH 13] - Loss: 1.1019190549850464 - F1: 0.4676277802327294
[EPOCH 14] - Loss: 1.0996673107147217 - F1: 0.4658408408408409
[EPOCH 15] - Loss: 1.0974055528640747 - F1: 0.46417486338797814
